In [ ]:
### Import dictionaries and read text (pre-processed)

import numpy as np
import pickle
import nltk
from HMM import unsupervised_HMM

from HMM_helper import (
    parse_observations, states_to_wordclouds, animate_emission,
    sample_sentence, obs_map_reverser)

#nltk.download('punkt')

File=open('Dictionaries/SyllDict.pkl','rb')
SyllDict=pickle.load(File)
File.close()

File=open('Dictionaries/LessThanEq.pkl','rb')
LessThanEq=pickle.load(File)
File.close()

File=open('Dictionaries/NoStress.pkl','rb')
NoStress=pickle.load(File)
File.close()

File=open('Dictionaries/Pronunciation.pkl','rb')
Pronunciation=pickle.load(File)
File.close()

File=open('Dictionaries/Rhyme.pkl','rb')
Rhyme=pickle.load(File)


File.close()

with open ("shakespeare_2.txt", "r") as myfile:
    data=myfile.readlines()

In [ ]:
# Identify words per line to create observation map

import re

for i in range(len(data)):
    #data[i] = data[i].split()
    data[i] = re.findall(r"[\w']+|[.,?;]", data[i])


obs_counter = 0
obs = []
obs_map = {}

for line in data:
    obs_elem = []
    for word in line:
        word = word.lower()
        if word not in obs_map:
                # Add unique words to the observations map.
            obs_map[word] = obs_counter
            obs_counter += 1
            
            # Add the encoded word.
        obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
    obs.append(obs_elem)

In [ ]:
# Train some HMMs


hmm2 = unsupervised_HMM(obs, 2, 20)
hmm3 = unsupervised_HMM(obs, 3, 20)
hmm5 = unsupervised_HMM(obs, 5, 20)
hmm8 = unsupervised_HMM(obs, 8, 20)
hmm12 = unsupervised_HMM(obs, 12, 20)

In [ ]:
# Generate random sonnet
# Because some words have apostrophes attached and dictionaries can't handle it, this may take a few tries


emission = []
states = []

L = hmm12.L
D = hmm12.D     

O = np.array(hmm12.O)
A = np.array(hmm12.A)

obs_map_r = obs_map_reverser(obs_map)

punctuation_to_remove = ['.', ',', ';', ':', '?', '!']


for k in range(0,14):
    emission = []
    states = []
    syll_count = 0

    # Initial state
    states.append(np.random.randint(0,L))
    emission.append(np.random.choice(D,None, True, O[states[0],:]))
    syll_count += SyllDict[obs_map_r[emission[0]]]['normal'][0]

    i = 1
    while syll_count < 10:
        states.append(np.random.choice(L,None, True, A[states[i-1],:]))
        emission.append(np.random.choice(D,None, True, O[states[i],:]))
        
        if obs_map_r[emission[i]] not in punctuation_to_remove:  
            syll_count += SyllDict[obs_map_r[emission[i]]]['normal'][0]
        i += 1
    sentence = [obs_map_r[i] for i in emission]
    print(' '.join(sentence).capitalize())
    
    if (k == 3 or k == 7 or k == 11):
        print('\n')

In [ ]:
# Generate random sonnet with rhymes
# Because some words have apostrophes attached and dictionaries can't handle it, this may take a few tries

emission = []
states = []

L = hmm12.L
D = hmm12.D     

O = np.array(hmm12.O)
A = np.array(hmm12.A)

obs_map_r = obs_map_reverser(obs_map)

punctuation_to_remove = ['.', ',', ';', ':', '?', '!']


for k in range(0,4):
    
    ### Line 1
    emission = []
    states = []
    syll_count = 0

    # Initial state
    states.append(np.random.randint(0,L))
    emission.append(np.random.choice(D,None, True, O[states[0],:]))
    syll_count += SyllDict[obs_map_r[emission[0]]]['normal'][0]
    rim_line1 = emission[0]
    i = 1
    while syll_count < 10:
        states.append(np.random.choice(L,None, True, A[:,states[i-1]]/np.sum(A[:,states[i-1]])))
        emission.append(np.random.choice(D,None, True, O[states[i],:]))
        
        if obs_map_r[emission[i]] not in punctuation_to_remove:  
            syll_count += SyllDict[obs_map_r[emission[i]]]['normal'][0]
        i += 1
    sentence = [obs_map_r[i] for i in emission]
    sentence_cop = np.copy(sentence)
    for kk in range(len(sentence)):
        sentence[len(sentence)-kk-1] = sentence_cop[kk]
    print(' '.join(sentence).capitalize())
    
    ### Line 2
    
    
    emission = []
    states = []
    syll_count = 0

    # Initial state
    states.append(np.random.randint(0,L))
    emission.append(np.random.choice(D,None, True, O[states[0],:]))
    syll_count += SyllDict[obs_map_r[emission[0]]]['normal'][0]
    rim_line2 = emission[0]
    i = 1
    while syll_count < 10:
        states.append(np.random.choice(L,None, True, A[:,states[i-1]]/np.sum(A[:,states[i-1]])))
        emission.append(np.random.choice(D,None, True, O[states[i],:]))
        
        if obs_map_r[emission[i]] not in punctuation_to_remove:  
            syll_count += SyllDict[obs_map_r[emission[i]]]['normal'][0]
        i += 1
    sentence = [obs_map_r[i] for i in emission]
    sentence_cop = np.copy(sentence)
    for kk in range(len(sentence)):
        sentence[len(sentence)-kk-1] = sentence_cop[kk]
    print(' '.join(sentence).capitalize())
    
    ### Line 3

    emission = []
    states = []
    syll_count = 0
    
    # Initial state
    states.append(np.random.randint(0,L))
    emission.append(np.random.choice(D,None, True, O[states[0],:]))
    while obs_map_r[emission[0]] not in Rhyme[obs_map_r[rim_line1]]:
        emission = []
        states = []
        states.append(np.random.randint(0,L))
        emission.append(np.random.choice(D,None, True, O[states[0],:]))

    i = 1
    while syll_count < 10:
        states.append(np.random.choice(L,None, True, A[:,states[i-1]]/np.sum(A[:,states[i-1]])))
        emission.append(np.random.choice(D,None, True, O[states[i],:]))
        
        if obs_map_r[emission[i]] not in punctuation_to_remove:  
            syll_count += SyllDict[obs_map_r[emission[i]]]['normal'][0]
        i += 1
    sentence = [obs_map_r[i] for i in emission]
    sentence_cop = np.copy(sentence)
    for kk in range(len(sentence)):
        sentence[len(sentence)-kk-1] = sentence_cop[kk]
    print(' '.join(sentence).capitalize())
    
    
    ### Line 4

        
    emission = []
    states = []
    syll_count = 0
    
    # Initial state
    states.append(np.random.randint(0,L))
    emission.append(np.random.choice(D,None, True, O[states[0],:]))
    while obs_map_r[emission[0]] not in Rhyme[obs_map_r[rim_line2]]:
        emission = []
        states = []
        states.append(np.random.randint(0,L))
        emission.append(np.random.choice(D,None, True, O[states[0],:]))

    i = 1
    while syll_count < 10:
        states.append(np.random.choice(L,None, True, A[:,states[i-1]]/np.sum(A[:,states[i-1]])))
        emission.append(np.random.choice(D,None, True, O[states[i],:]))
        
        if obs_map_r[emission[i]] not in punctuation_to_remove:  
            syll_count += SyllDict[obs_map_r[emission[i]]]['normal'][0]
        i += 1
    sentence = [obs_map_r[i] for i in emission]
    sentence_cop = np.copy(sentence)
    for kk in range(len(sentence)):
        sentence[len(sentence)-kk-1] = sentence_cop[kk]
    print(' '.join(sentence).capitalize())
    
    print('\n')

In [ ]:
## Visualize states
wordclouds = states_to_wordclouds(hmm12, obs_map,10)

In [ ]:
## Animate emission generation
anim = animate_emission(hmm12, obs_map, M=8)
HTML(anim.to_html5_video())